In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import requests

files = [
    "SYB65_1_202209_Population, Surface Area and Density.csv",
    "SYB65_246_202209_Population Growth, Fertility and Mortality Indicators.csv",
    "SYB65_327_202209_International Migrants and Refugees.csv"
]

urls = [
    "https://data.un.org/_Docs/SYB/CSV/SYB65_1_202209_Population,%20Surface%20Area%20and%20Density.csv",
    "https://data.un.org/_Docs/SYB/CSV/SYB65_246_202209_Population%20Growth,%20Fertility%20and%20Mortality%20Indicators.csv",
    "https://data.un.org/_Docs/SYB/CSV/SYB65_327_202209_International%20Migrants%20and%20Refugees.csv",
]

for url, file in zip(urls, files):
    r = requests.get(url, allow_redirects=True)
    open(file, "wb").write(r.content)

In [ ]:
if not os.path.exists("population"):
    os.makedirs("population")

In [ ]:
tables = {}

for file in files:
    df = pd.read_csv(file, header=None, encoding='latin-1')
    groups = df.groupby(df.columns[1])
    for name, group in groups:
        country_df = pd.DataFrame(group)
        country_df[4] = country_df[4].str.replace(",", "")
        country_df = country_df.drop([0, 1, 5, 6], axis=1)
        country_df = country_df.pivot_table(4, [3], 2)
        country_df = country_df.reset_index()
        country_df = country_df.rename(columns={3: "attribute"})
        if name in tables:
            tables[name].append(country_df)
        else:
            tables[name] = [country_df]

In [ ]:
tables_copy = tables
for name in tables_copy:
    country_df = tables_copy[name].pop()
    while len(tables_copy[name]) > 0:
        country_df = pd.merge(country_df, tables_copy[name].pop(), how="outer")
    country_df.to_csv(f"./population/{name}.csv", index=False)

In [ ]:
india = pd.read_csv("./population/India.csv")
india

In [ ]:
for index, row in india.iterrows():
    plt.figure()

    label = row[0]
    values = row[1:]
    
    plt.bar(values.index, values)
    plt.title(label)
    
    plt.show()